In [1]:
%load_ext autoreload
%autoreload 2

In [102]:
from qiskit import execute, Aer, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator, UnitarySimulator
import numpy as np

In [103]:
beta = 0.5 * acos(1/sqrt(3))
T0 = [np.cos(beta), np.exp(1j*np.pi/4)*np.sin(beta)]
T0

[(0.8880738339771153-0j), (0.3250575836718681+0.32505758367186804j)]

In [72]:
qc = QuantumCircuit(1)
qc.initialize(T0)
qc.decompose().decompose().decompose().draw()

global phase: π/8
          ┌─────────────┐┌─────────┐
q_0: ─|0>─┤ Ry(0.95532) ├┤ Rz(π/4) ├
          └─────────────┘└─────────┘

In [140]:
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.tdg(0)
qc.t(0) # use magic state
qc.h(0)
qc.measure(0, 0)
qc.draw()

┌───┐┌─────┐┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ Tdg ├┤ T ├┤ H ├┤M├
     └───┘└─────┘└───┘└───┘└╥┘
c: 1/═══════════════════════╩═
                            0

In [149]:
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.t(0)
backend = StatevectorSimulator()
results = backend.run(qc).result()
print(results.get_statevector())

[0.70710678+0.j  0.5       +0.5j]


In [165]:
qc = QuantumCircuit(2, 2)
qc.x(0)
qc.measure(range(2), range(2))
backend = QasmSimulator()
results = backend.run(qc, shots=10, memory=True).result()
print(results.get_memory())

['01', '01', '01', '01', '01', '01', '01', '01', '01', '01']


In [173]:
def get_magic_state_circuit():
    N = 5
    qc = QuantumCircuit(N, N)
    for i in range(N):
        qc.ry(0.95532, i)
        qc.rz(pi/4, i)
    #     qc.initialize(T0,qubits=i)
    qc.cz(4, range(3))
    qc.barrier()
    qc.cz(3, [0, 2])
    qc.cy(3, 4)
    qc.barrier()
    qc.cz(2, [0, 3])
    qc.cy(2, 4)
    qc.barrier()
    qc.cz(1, 0)
    qc.cy(1, 4)
    qc.barrier()
    qc.cz(0, 3)
    qc.cx(0, 4)
    qc.barrier()
    qc.h(range(4))
    qc.z(4)
    qc.barrier()
#     qc.reverse_bits()
    qc.measure(range(4), range(4))
    qc.save_density_matrix([4], pershot=True, conditional=True)
    return qc

qc = get_magic_state_circuit()
qc.draw()

┌─────────────┐┌─────────┐          ░             ░             ░         »
q_0: ┤ Ry(0.95532) ├┤ Rz(π/4) ├─■────────░──■──────────░──■──────────░──■──────»
     ├─────────────┤├─────────┤ │        ░  │          ░  │          ░  │      »
q_1: ┤ Ry(0.95532) ├┤ Rz(π/4) ├─┼──■─────░──┼──────────░──┼──────────░──■───■──»
     ├─────────────┤├─────────┤ │  │     ░  │          ░  │          ░      │  »
q_2: ┤ Ry(0.95532) ├┤ Rz(π/4) ├─┼──┼──■──░──┼──■───────░──■──■───■───░──────┼──»
     ├─────────────┤├─────────┤ │  │  │  ░  │  │       ░     │   │   ░      │  »
q_3: ┤ Ry(0.95532) ├┤ Rz(π/4) ├─┼──┼──┼──░──■──■───■───░─────■───┼───░──────┼──»
     ├─────────────┤├─────────┤ │  │  │  ░       ┌─┴─┐ ░       ┌─┴─┐ ░    ┌─┴─┐»
q_4: ┤ Ry(0.95532) ├┤ Rz(π/4) ├─■──■──■──░───────┤ Y ├─░───────┤ Y ├─░────┤ Y ├»
     └─────────────┘└─────────┘          ░       └───┘ ░       └───┘ ░    └───┘»
c: 5/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░          ░ ┌───┐ ░ ┌─┐            
«q_0: ─░──■───■───░─┤ H ├─░─┤M├────────────
«      ░  │   │   ░ ├───┤ ░ └╥┘┌─┐         
«q_1: ─░──┼───┼───░─┤ H ├─░──╫─┤M├─────────
«      ░  │   │   ░ ├───┤ ░  ║ └╥┘┌─┐      
«q_2: ─░──┼───┼───░─┤ H ├─░──╫──╫─┤M├──────
«      ░  │   │   ░ ├───┤ ░  ║  ║ └╥┘┌─┐   
«q_3: ─░──■───┼───░─┤ H ├─░──╫──╫──╫─┤M├───
«      ░    ┌─┴─┐ ░ ├───┤ ░  ║  ║  ║ └╥┘ ░ 
«q_4: ─░────┤ X ├─░─┤ Z ├─░──╫──╫──╫──╫──░─
«      ░    └───┘ ░ └───┘ ░  ║  ║  ║  ║  ░ 
«c: 5/═══════════════════════╩══╩══╩══╩════
«                            0  1  2  3

In [151]:
for _ in range(10):
    backend = StatevectorSimulator()
    results = backend.run(qc).result()
    psi = results.get_statevector()[15]
    if np.nonzero(psi):
        print(psi)

0j
0j
0j
0j
0j
0j
0j
0j
0j
0j


In [152]:
#00001 10000
print(results.get_statevector())

[ 0.        +0.j          0.        +0.j          0.        +0.j
 -0.43819446-0.83804706j  0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        -0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.        +0.j         -0.32227697+0.04242993j -0.        +0.j
 -0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]


In [153]:
backend = QasmSimulator(method='density_matrix')
results = backend.run(qc, shots=10, memory=True).result()
# results.get_counts()
# results.get_statevector()

In [154]:
results.get_memory()

['01110',
 '01111',
 '01111',
 '01110',
 '00111',
 '00011',
 '00001',
 '01100',
 '00001',
 '00011']

In [155]:
results.to_dict()

{'backend_name': 'qasm_simulator',
 'backend_version': '0.8.2',
 'qobj_id': 'f3d956a0-dc8f-49c4-bf60-5bb87502e418',
 'job_id': '4ef95ff4-0f06-4989-a8ee-646fa6269842',
 'success': True,
 'results': [{'shots': 10,
   'success': True,
   'data': {'counts': {'0xc': 1,
     '0x1': 2,
     '0x7': 1,
     '0x3': 2,
     '0xe': 2,
     '0xf': 2},
    'memory': ['0xe',
     '0xf',
     '0xf',
     '0xe',
     '0x7',
     '0x3',
     '0x1',
     '0xc',
     '0x1',
     '0x3'],
    'density_matrix': {'0x0': [array([[ 0.53207425+3.85185989e-34j, -0.09622435+9.62243545e-02j],
             [-0.09622435-9.62243545e-02j,  0.46792575+9.14816724e-34j]]),
      array([[ 0.53207425+3.85185989e-34j, -0.09622435+9.62243545e-02j],
             [-0.09622435-9.62243545e-02j,  0.46792575+9.14816724e-34j]]),
      array([[ 0.53207425+3.85185989e-34j, -0.09622435+9.62243545e-02j],
             [-0.09622435-9.62243545e-02j,  0.46792575+9.14816724e-34j]]),
      array([[ 0.53207425+3.85185989e-34j, -0.09622435+9.62